In [51]:
import pandas as pd
import json
import numpy as np
import fr_core_news_sm
import nltk
from nltk.stem.snowball import SnowballStemmer
from tqdm.notebook import tqdm

from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

#nltk.download()

In [2]:
fd = "../0. Data/"


def parse_obj(obj):
    for key in obj:
        if isinstance(obj[key], str):
            obj[key] = obj[key].encode('latin_1').decode('utf-8')
        elif isinstance(obj[key], list):
            obj[key] = list(map(lambda x: x if type(x) != str else x.encode('latin_1').decode('utf-8'), obj[key]))
        pass
    return obj


In [3]:
file = open(fd + 'message_1.json')
data = json.load(file, object_hook=parse_obj)
df = pd.json_normalize(data['messages'])
for i in np.arange(2,6) : 
    file = open(fd + 'message_'+str(i)+'.json', encoding='utf8')
    data = json.load(file)
    df_temp = pd.json_normalize(data['messages'])
    df=df.append(df_temp)
    

df['date_time']=pd.to_datetime(df['timestamp_ms'], unit='ms') 
df['content']=df['content'].str.lower()
df.drop(columns=['timestamp_ms','gifs','is_unsent','photos','type','videos','audio_files','sticker.uri',
                 'call_duration','share.link','share.share_text','users','files'],inplace=True)

df['year']=df['date_time'].dt.year
df=df[df['year']==2021]

In [45]:
df=df[~df['content'].isna()].reset_index()


In [46]:
df.sample(5)

,index,sender_name,reactions,content,date_time,year
5663,7609,Robin Goutard,NaN,bière en ville putain il fait beau ahlalaaaa,2021-05-21 12:43:55.984,2021
2381,3751,Thomas Liberge,NaN,l'ex d'harry,2021-08-24 11:23:51.173,2021
4034,5733,Alex Dns,NaN,nan mais quelle soirée de merde,2021-06-28 22:20:33.718,2021
9001,1593,Robin Goutard,NaN,aucune chance ahahah,2021-02-26 10:17:22.143,2021
3899,5556,Alex Dns,NaN,décidément c'est vraiment la fête en ce moment,2021-07-06 06:40:37.873,2021


In [49]:
nlp = fr_core_news_sm.load()

temmenized = pd.DataFrame(columns=['sender_name','stem'])
for i in tqdm(np.arange(df.shape[0]-1)) :
    doc = nlp(df['content'][i])
    for token in doc:
        dict = {'sender_name': df['sender_name'][i], 'stem':token.lemma_}
        temmenized=temmenized.append(dict, ignore_index = True)
            

  0%|          | 0/10628 [00:00<?, ?it/s]

In [118]:
temmenized_root=temmenized[~temmenized['stem'].isin([fr_stop,'.','le','de','être','je','avoir','ce','un','!','?',',','pas',
                                                     'en','on','et','que','cela','©','haha','il','🤣','ahah','à','pour','te','c’',
                                                     'faire','mais','sur','aller','tu','mon','qui','ã','avec','au','vous',
                                                    ';)','tout','y','plus','j’','dans','bon','me',')','(','moi','se','...','^^',
                                                    'non'])]





Series([], dtype: int64)

In [121]:
temmenized_root[temmenized_root['sender_name']=='Alex Dns'].value_counts().head(10)

sender_name  stem   
Alex Dns     bien       77
             si         61
             comme      58
             pouvoir    54
             lui        49
             dire       48
             petit      48
             voir       44
                        42
             "          42
dtype: int64

In [89]:
temmenized_root[temmenized_root['sender_name']=='Robin Goutard'].value_counts().head(10)

{'a',
 'abord',
 'afin',
 'ah',
 'ai',
 'aie',
 'ainsi',
 'ait',
 'allaient',
 'allons',
 'alors',
 'anterieur',
 'anterieure',
 'anterieures',
 'antérieur',
 'antérieure',
 'antérieures',
 'apres',
 'après',
 'as',
 'assez',
 'attendu',
 'au',
 'aupres',
 'auquel',
 'aura',
 'auraient',
 'aurait',
 'auront',
 'aussi',
 'autre',
 'autrement',
 'autres',
 'autrui',
 'aux',
 'auxquelles',
 'auxquels',
 'avaient',
 'avais',
 'avait',
 'avant',
 'avec',
 'avoir',
 'avons',
 'ayant',
 'bas',
 'basee',
 'bat',
 "c'",
 'car',
 'ce',
 'ceci',
 'cela',
 'celle',
 'celle-ci',
 'celle-la',
 'celle-là',
 'celles',
 'celles-ci',
 'celles-la',
 'celles-là',
 'celui',
 'celui-ci',
 'celui-la',
 'celui-là',
 'cent',
 'cependant',
 'certain',
 'certaine',
 'certaines',
 'certains',
 'certes',
 'ces',
 'cet',
 'cette',
 'ceux',
 'ceux-ci',
 'ceux-là',
 'chacun',
 'chacune',
 'chaque',
 'chez',
 'ci',
 'cinq',
 'cinquantaine',
 'cinquante',
 'cinquantième',
 'cinquième',
 'combien',
 'comme',
 'comment',